In [1]:
from whispr import *

In [2]:
# gene expression. 3ul rxns, 75% txtl
source_plate_type = '384PP_AQ_BP' 

sp_plasmids_file = 'source_plates/221004_arpae_plasmids.xlsx'
sp_plasmids = pd.read_excel(sp_plasmids_file, index_col = 0)
sp_plasmids = sp_plasmids[~sp_plasmids['Well'].isna()]

filename = '221004_arpae_genex.csv'
layout_genex_file = 'plate_layouts/'+filename 
layout_genex = pd.read_csv(layout_genex_file, index_col = 0, dtype = str)

mt_genex_file = 'mixing_tables/221004_arpae_plasmids.csv'
mt_genex = pd.read_csv(mt_genex_file, index_col = 0, dtype = str).fillna(0)

checkInputs(sp_plasmids, mt_genex,source_plate_type)

vol_table_df = generateVolumeTable(mt_genex, sp_plasmids, rxn_vol = 0.25*3, total_vol = 3, fill_with='water')

#specify rxn_vol (default = 2.5) and total_vol (default = 10) if you'd like to change the volume of each individual replicate or the total reaction volume
protocol_genex_df = writeProtocol(source_plate_type, vol_table_df, layout_genex,sp_plasmids, update_source_vol=False)


Volumes used from each well for this protocol:
{'1_A1': 9.8, '1_P1': 0.6, '1_P2': 0.8, '1_P3': 1.2, '1_P4': 1.85}


In [3]:
# everything gets txtl
txtl_vol = 3*0.75*1000
rxns = len(protocol_genex_df['Destination Well'].unique())
print('Need '+str(rxns*txtl_vol/1000)+' uL txtl')
txtl_well = 'P24'

for dwell in protocol_genex_df['Destination Well'].unique():
    row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': txtl_well,
                                'Destination Plate Name':'Destination[1]', 'Destination Well': dwell, 'Transfer Volume': txtl_vol}
    protocol_genex_df = protocol_genex_df.append(row, ignore_index = True)

protocol_genex_df.to_csv('protocols/'+filename,index = False)

# carry out gene expression reaction

Need 42.75 uL txtl


In [4]:
# dilution. everything gets Tris, for 20x -> 57uL
protocol_dilution_df = pd.DataFrame(columns=protocol_genex_df.columns)
source_plate_type = '6RES_AQ_B' 
tris_well = 'A1'
tris_vol = 57*1000

for dwell in protocol_genex_df['Destination Well'].unique():
    row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': tris_well,
                                'Destination Plate Name':'Destination[1]', 'Destination Well': dwell, 'Transfer Volume': tris_vol}
    protocol_dilution_df = protocol_dilution_df.append(row, ignore_index = True)

protocol_dilution_df.to_csv('protocols/221004_arpae_dilutions.csv',index = False)

In [5]:
# from source plate with buffer, mix into 96

# from source plate with genex, mix the diluted txtl

In [6]:
# biosynthesis. 25ul rxns, 2.5ul of diluted txtl

# buffer source plate
sp_buffers_file = 'source_plates/221004_arpae_buffers.xlsx'
sp_buffers= pd.read_excel(sp_buffers_file, index_col = 0)
sp_buffers = sp_buffers[~sp_buffers['Well'].isna()]

# HEPES reservoir 
sp_hepes = pd.DataFrame(columns=sp_buffers.columns)
sp_hepes = sp_hepes.append({'Label':'HEPES', 'Well':'A1,A2', 'Concentration':'', 'Volume':'2000,2000'}, ignore_index = True)

# gene expression source plate (diluted w/ Tris)
sp_genex = sp_from_layout(layout_genex, 60)

# group source plates
sp_types = ['384PP_AQ_BP','6RES_AQ_BP','384PP_AQ_BP'] # triple check this
sps = [sp_buffers,sp_hepes,sp_genex]

# get mixing table
mt_biosyn_file = 'mixing_tables/221004_arpae_buffers.csv'
mt_biosyn = pd.read_csv(mt_biosyn_file, index_col = 0, dtype = str).fillna(0)

# check formats
checkInputs(sps,mt_biosyn,sp_types)

# layouts for destination plate(s)
filename = '221004_arpae_biosyn_pos.csv'
layout_biosyn1_file = 'plate_layouts/'+filename 
layout_biosyn1 = pd.read_csv(layout_biosyn1_file, index_col = 0, dtype = str)
filename = '221004_arpae_biosyn_neg.csv'
layout_biosyn2_file = 'plate_layouts/'+filename 
layout_biosyn2 = pd.read_csv(layout_biosyn2_file, index_col = 0, dtype = str)
layouts = [layout_biosyn1,layout_biosyn2]

vol_table_df = generateVolumeTable(mt_biosyn, sps, rxn_vol = 22.5, total_vol = 25, fill_with='HEPES')

protocol_biosyn_df = writeProtocol(sp_types, vol_table_df, layouts,sps, update_source_vol=False)

protocol_biosyn_df.to_csv('protocols/221004_arpae_biosyn.csv',index = False)

Volumes used from each well for this protocol:
{'1_A1': 47.4, '1_A2': 17.4, '1_B1': 47.725, '1_B2': 1.5, '1_C1': 46.8, '1_C2': 46.2, '1_C3': 46.8, '1_C4': 46.2, '1_D1': 27.0, '1_E1': 47.5, '1_E2': 20.0, '1_F1': 2.7, '1_G1': 2.7, '1_H1': 47.8, '1_H2': 16.25, '1_I1': 47.25, '1_I2': 33.75, '1_J1': 47.8, '1_J2': 0.5, '1_K1': 25.35, '1_L1': 0.45, '2_A1': 1742.1, '2_A2': 98.825, '3_A1': 2.5, '3_A2': 42.5, '3_B2': 42.5, '3_C2': 42.5, '3_D2': 7.5, '3_B1': 2.5, '3_C1': 2.5, '3_D1': 2.5, '3_E1': 2.5, '3_F1': 2.5, '3_F2': 42.5, '3_G2': 42.5, '3_H2': 42.5, '3_I2': 7.5, '3_G1': 2.5, '3_H1': 2.5, '3_I1': 2.5, '3_J1': 2.5, '3_K1': 12.5}
